Από την Ομάδα 70, που αποτελείται από τους:
* Ιωάννης Μιχαήλ Καζελίδης 03117885
* Μάριος Κερασιώτης 03117890
* Ιωάννης Γκιορτζής 03117152

Στην ομάδα μας αντιστοιχούν τα εξής datasets:
* U09: Connectionist Bench (Sonar, Mines vs. Rocks)
* K02: Company Bankruptcy Prediction

# UCI: Connectionist Bench (Sonar, Mines vs. Rocks)

In [ ]:
# !pip3 install -q -U pip matplotlib numpy pandas seaborn scikit-learn

In [ ]:
DATASET_URI = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data'
PICKLE_FILE = 'sonar.all-data.df'

import os.path
import pickle
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tabulate
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from IPython.display import HTML, display
from sklearn.decomposition import PCA
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import (
    SelectKBest,
    SelectPercentile,
    VarianceThreshold,
    chi2,
    f_classif,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
)
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, Normalizer, RobustScaler, StandardScaler

%matplotlib inline
sns.set(style='whitegrid', palette="cubehelix")
sns.set(rc={'figure.figsize': (20, 15)})
pd.options.display.max_columns = None

warnings.filterwarnings("ignore")

## Εισαγωγή και επισκόπηση

### Διερεύνηση του Dataset

In [ ]:
if os.path.isfile(PICKLE_FILE):
    with open(PICKLE_FILE, 'rb') as f:
        data = pickle.load(f)
else:
    column_headers = ["T{}".format(str(i)) for i in range(60)]
    column_headers.append("Object")
    data = pd.read_csv(DATASET_URI, names=column_headers)

    with open(PICKLE_FILE, 'wb') as f:
        pickle.dump(data, f)
data

In [ ]:
data.info()

In [ ]:
features = data[data.columns.drop('Object')]
labels = data['Object'].astype('category')
del (data)

In [ ]:
print("Dimensions of features:", features.shape)
print("Dimensions of labels:", labels.shape)

In [ ]:
print('Unique labels', np.unique(labels))
labels.value_counts() / len(labels)

In [ ]:
features.describe()

In [ ]:
boxplot = sns.boxplot(data=features)
boxplot.set(title='Boxplot of dataset features',
            xlabel='Column',
            ylabel='Value')
sns.despine(offset=2)
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.heatmap(features.corr())
plt.show()

### Απάντηση στα ζητούμενα

1. Σύντομη Παρουσίαση Dataset και Περιγραφή Προβλήματος
     - Το γενικό πρόβλημα αποτελεί την κατηγοριοποίηση διαφόρων σημάτων με την βοήθεια ενός ταξινομητή. Ένα σύστημα SO.N.A.R. εκπέμπει σήματα, για τα οποία καλούμαστε να διακρίνουμε αν η επιφάνεια στην οποία ανακλώνται είναι μέταλλο ή πέτρα.

     - Το συγκεκριμένο dataset είναι ένα σύνολο μοτίβων από σήματα που λήφθηκαν από μεταλλικούς και πέτρινους κυλίνδρους. Κάθε γραμμή αποτελεί είτε έναν μεταλλικό είτε ένα πέτρινο κύλινδρο και οι μετρήσεις κατά μήκος της γραμμής είναι οι διάφορες ενέργειες των σημάτων σε διαφορετικές γωνίες και καταστάσεις του. Κάθε γραμμή περιέχει 60 μετρήσεις, καθώς και τον τύπο του κυλίνδρου στην 61η στήλη. Συνολικά υπάρχουν 208 τέτοιες γραμμές από τα διάφορα υλικά.  

2. Χρειάστηκε να κάνετε μετατροπές στα αρχεία plain text για την εισαγωγή του; αν ναι, ποιες είναι αυτές;
   - Δεν χρειάστηκε κάποια μετατροπή στα δεδομένα εκτός από το να διαβάζουμε το αρχείο σε μορφή csv (με την επιλογή να μην έχει το dataset κεφαλίδες).

3. Δώστε το πλήθος δειγμάτων και χαρακτηριστικών, και το είδος όλων των χαρακτηριστικών. Υπάρχουν μη διατεταγμένα χαρακτηριστικά και ποια είναι αυτά;
    - Συνολικά έχουμε 208 δείγματα. Από αυτά, τα 111 είναι μοτίβα τα οποία λαμβάνουμε από τους μεταλλικούς κυλίνδρους σε διαφορετικές γωνίες και καταστάσεις, ενώ τα 97 είναι μοτίβα τα οποία λαμβάνουμε από τις κυλινδρικές πέτρες κάτω από αντίστοιχες συνθήκες. Επίσης, για κάθε ένα από αυτά τα δείγματα, έχουμε 60 μετρήσεις, οι οποίες αναφέρονται σε ενέργεια, της οποίας η τιμή κυμαίνεται κάθε φορά από 0.0 έως 1.0 (στο dataset αυτές αναπαριστώνται ως float64), καθώς και ένα αναγνωριστικό που δείχνει την κλάση στην οποία ανήκει το δείγμα, με M για μέταλλο και R για πέτρα (στο dataset αυτό αναπαριστάται ως object). Αυτά τα "αναγνωριστικά" είναι και τα χαρακτηριστικά τα οποία είναι μη διατεταγμένα, εφόσον δεν είναι αριθμητικά, και έτσι δεν μπορούμε να τα διατάξουμε με κάποιο τρόπο.

4. Υπάρχουν επικεφαλίδες; Αρίθμηση γραμμών;
    - Το αρχείο αποτελείται μόνο από τα δεδομένα διαχωρισμένα με το σύμβολο `,` και δεν περιέχει ούτε κεφαλίδες ούτε αρίθμηση γραμμών. Κεφαλίδες προσθέσαμε εμείς ύστερα στο dataframe.

5. Ποιες είναι οι ετικέτες των κλάσεων και σε ποια κολόνα βρίσκονται;
    - Οι ετικέτες βρίσκονται στην στήλη 61 και έχουν τις τιμές `R` και `Μ` για πέτρα ή μέταλλο αντίστοιχα.

6. Υπάρχουν απουσιάζουσες τιμές; Πόσα είναι τα δείγματα με απουσιάζουσες τιμές και ποιο το ποσοστό τους επί του συνόλου;
    - Κάθε γραμμή έχει 60 μετρήσεις και ένα αναγνωριστικό που δείχνει την κλάση στην οποία ανήκει και έτσι δεν έχουμε απουσιάζουσες τιμές για κανένα από τα δείγματα (όπως φαίνεται και παραπάνω από τον κώδικα). Έτσι το ποσοστό των δειγμάτων με απουσιάζουσες τιμές είναι `0%`.

7. Ποιο είναι το πλήθος των κλάσεων και τα ποσοστά δειγμάτων τους επί του συνόλου; Αν θεωρήσουμε ότι ένα dataset είναι μη ισορροπημένο αν μια οποιαδήποτε κλάση είναι 1.5 φορά πιο συχνή από κάποια άλλη (60%-40% σε binary datasets) εκτιμήστε αν το dataset είναι ισορροπημένο ή όχι.
    - Το πλήθος των κλάσεων είναι 2, μεταλλικοί κύλινδροι και κυλινδρικές πέτρες. Τα ποσοστά δειγμάτων τους επί του συνόλου είναι αντίστοιχα #Μ = 111, και άρα `53.37%` και #R = 97 και άρα `46.63%`. Καμία κλάση δεν είναι 1.5 φορά πιο συχνή από κάποια άλλη, και συνεπώς το dataset είναι ισορροπημένο.

## Προετοιμασία

### Διαχείριση κατηγορικών δεδομένων

Καταρχάς, διαχωρίζουμε το σύνολο δεδομένων σε σύνολο εκπαίδευσης (train set) και σύνολο (test set) με 30% των δειγμάτων στο test set και στην συνέχεια θα κάνουμε διαχείριση κατηγορικών δεδομένων, όπως φαίνεται παρακάτω.

In [ ]:
train, test, train_labels, test_labels = train_test_split(features,
                                                          labels,
                                                          test_size=0.3)

In [ ]:
train_labels

In [ ]:
le = LabelEncoder()
# we are fitting the Label Encoder with train_labels
# to avoid data leakage
le.fit(train_labels)
train_labels = pd.Series(le.transform(train_labels),
                         name='Object').astype('category')
test_labels = pd.Series(le.transform(test_labels),
                        name='Object').astype('category')

In [ ]:
train_labels

Όπως φαίνεται και παραπάνω, το κατηγορικό (και μη διατεταγμένο) χαρακτηριστικό που διέκρινε τα δείγματα σε R και M, μέσω του label encoder τα αντιστοιχήσαμε σε 1 και 0 αντίστοιχα, για να είναι πιο εύκολη η επεξεργασία και η διαχείριση των δεδομένων μας και να είναι αποδοτικότερο το πρόγραμμά μας.

### Διαχείριση απουσιάζουσων τιμών

Τα δεδομένα δεν έχουν απουσιάζουσες τιμές και έτσι δεν χρειάζεται κάποια περαιτέρω προετοιμασία του dataset.

## Ταξινόμηση

Αρχικά θα δούμε πώς συμπεριφέρονται οι ταξινομητές χωρίς καμία βελτιστοποίηση (out-of-the-box) και με όλες τις παραμέτρους σε default τιμές.

In [ ]:
out_of_the_box_scores = pd.DataFrame()


def add_scores(metric, classifier, score):
    global out_of_the_box_scores
    out_of_the_box_scores = out_of_the_box_scores.append(
        {
            'Metric': metric,
            'Classifier': classifier,
            'Score': score
        },
        ignore_index=True)

### Dummy classifier

In [ ]:
dc = DummyClassifier()
dc.fit(train, train_labels)
preds = dc.predict(test)

score_acc = accuracy_score(test_labels, preds)
score_f1 = f1_score(test_labels, preds)

add_scores('Accuracy', 'Dummy', score_acc)
add_scores('F1', 'Dummy', score_f1)

### Gaussian Naive Bayes (GNB) classifier

In [ ]:
gnb = GaussianNB()
gnb.fit(train, train_labels)
preds = gnb.predict(test)

score_acc = accuracy_score(test_labels, preds)
score_f1 = f1_score(test_labels, preds)

add_scores('Accuracy', 'Gaussian Naive Bayes', score_acc)
add_scores('F1', 'Gaussian Naive Bayes', score_f1)

### KNeirestNeighbors (kNN) classifier

In [ ]:
knn = KNeighborsClassifier()
knn.fit(train, train_labels)
preds = knn.predict(test)

score_acc = accuracy_score(test_labels, preds)
score_f1 = f1_score(test_labels, preds)

add_scores('Accuracy', 'K-nearest neighbors', score_acc)
add_scores('F1', 'K-nearest neighbors', score_f1)

### Logistic Regression (LR) classifier

In [ ]:
lr = LogisticRegression()
lr.fit(train, train_labels)
preds = lr.predict(test)

score_acc = accuracy_score(test_labels, preds)
score_f1 = f1_score(test_labels, preds)

add_scores('Accuracy', 'Logistic Regression', score_acc)
add_scores('F1', 'Logistic Regression', score_f1)

### Παρουσίαση επίδοσης out-of-the-box

In [ ]:
out_of_the_box_scores.groupby('Classifier').apply(
    lambda a: a.drop('Classifier', axis=1)[:])

In [ ]:
barplot = sns.barplot(data=out_of_the_box_scores,
                      y='Score',
                      x='Classifier',
                      hue='Metric')
barplot.set(title='Accuracy and F1 scores for out of the box classifiers')
plt.show()

Παραπάνω βλέπουμε την συμπεριφορά των ταξινομητών χωρίς καμία βελτιστοποίηση (out-of-the-box) και με όλες τις παραμέτρους σε default τιμές. Παρατηρούμε πως και για τις 2 μετρικές, οι επιδόσεις βελτιώνονται όσο περνάμε από τον Dummy στον Gaussian, και από τον Gaussian στον kNN, όπου και εμφανίζονται οι μεγαλύτερες επιδόσεις. Παρόλο που από τον kNN στον LR παρουσιάζεται μια πτώση και στις 2 μετρικές, η επίδοση του LR είναι καλύτερη στις 2 μετρικές και από τον Dummy αλλά και από τον Gaussian ταξινομητή. Αυτό σημαίνει πως χωρίς καμία βελτιστοποίηση κυρίως οι ταξινομητές kNN και LR εμφανίζουν την καλύτερη επίδοση στις υπό εξέταση μετρικές (με εμφανώς καλύτερη επίδοση στον kNN), ενώ ο Dummy παρουσιάζει την χαμηλότερη.

## Βελτιστοποίηση

### Ανάλυση Dataset

Με βάση το boxplot που φαίνεται παραπάνω παρατηρούμε πως θα βοηθούσε να κανονικοποιήσουμε τα δεδομένα μας. 

In [ ]:
scaler = StandardScaler()
robust_scaler = RobustScaler()

scaler.fit_transform(features)
robust_scaler.fit_transform(features)

scaled_features = pd.DataFrame(scaler.fit_transform(features),
                               columns=features.columns)

fig, ax = plt.subplots(1, 2)

sns.boxplot(data=scaled_features, ax=ax[0]).set(title='Standard Scaler',
                                                xlabel='Column',
                                                ylabel='Value')
sns.despine(offset=2)
plt.xticks(rotation=45)
sns.boxplot(data=robust_scaler.fit_transform(features),
            ax=ax[1]).set(title='Robust Scaler',
                          xlabel='Column',
                          ylabel='Value')
sns.despine(offset=2)
plt.xticks(rotation=45)
plt.show()

Πράγματι, με την χρήση των Scaler φαίνεται ότι τα δεδομένα κανονικοποιήθηκαν, οπότε μπορούμε να τους χρησιμοποιήσουμε στην προσπάθεια βελτίωσης των επιδόσεων των ταξινομητών.

### Βοηθητικές συναρτήσεις

In [ ]:
preprocessing_scores = pd.DataFrame()


def train_best_model(cv_classifier, classifier, scoring_metric):
    global preprocessing_scores
    out_of_the_box_score = lambda m: out_of_the_box_scores.loc[
        (out_of_the_box_scores['Classifier'] == classifier) &
        (out_of_the_box_scores['Metric'] == m)]['Score'].values[0]
    pipe = cv_classifier.best_estimator_

    start_time = time.time()
    pipe.fit(train, train_labels)
    train_time = time.time() - start_time

    start_time = time.time()
    preds = pipe.predict(test)
    test_time = time.time() - start_time

    score_acc = accuracy_score(test_labels, preds)
    score_f1 = f1_score(test_labels, preds)

    preprocessing_scores = preprocessing_scores.append(
        {
            "Classifier": classifier,
            "Scoring_metric": scoring_metric,
            "Accuracy": score_acc,
            "ΔAccuracy": score_acc - out_of_the_box_score('Accuracy'),
            "F1": score_f1,
            "ΔF1": score_f1 - out_of_the_box_score('F1'),
            "Train_time": train_time,
            "Test_time": test_time,
            "Preds": preds
        },
        ignore_index=True)


def use_cv(operations, scoring_metric, param_grid, classifier):
    pipe = Pipeline(steps=operations, memory='tmp')
    cv_classifier = GridSearchCV(pipe,
                                 param_grid,
                                 cv=10,
                                 scoring=scoring_metric,
                                 n_jobs=-1)
    cv_classifier.fit(train, train_labels)

    print('''Statistics for {} on {} scoring metric:
          Best Score = {}
          Best Estimator = {}'''.format(classifier, scoring_metric,
                                        cv_classifier.best_score_,
                                        cv_classifier.best_estimator_))
    return cv_classifier


def get_params(params_dicts, operations):
    params_dict = {}
    for (op_name, _) in operations:
        if op_name in params_dicts:
            for k, v in params_dicts[op_name].items():
                params_dict[op_name + '__' + k] = v
    return params_dict

Για την βελτιστοποίηση όλων των classifier χρησιμοποιήσαμε κάθε φορά ένα dictionary, το params_dicts, στο οποίο έχουμε τα operations και τις τιμές των υπερπαραμέτρων από τα οποία αντλούμε για να αποφανθούμε ποια θα οδηγήσουν στην βελτιστοποίηση του best score στο train set. Συγκεκριμένα, σε όλες τις περιπτώσεις κοινό έχουμε τον Robust Scaler με τις υπερπαραμέτρους with_centering, with_scaling και unit_variance, τον Standard Scaler, με τις υπερπαραμέτρους with_mean και with_std και το PCA, με την υπερπαράμετρο n_components να παίρνει τιμές από μια λίστα, όπως φαίνεται παρακάτω. Τέλος, χρησιμοποιήσαμε και το Select Percentile, το οποίο κάθε φορά επιλέγει τα χαρακτηριστικά με βάση ένα ποσοστό (που παίρνει από μια λίστα που φαίνεται παρακάτω) των καλύτερων scores. Καλό θα ήταν γενικά να αναφερθεί επίσης πως δοκιμάσαμε τόσο την χρήση του Random Oversampler όσο και του Variance Threshold, αλλά κανένα από τα 2 δεν φάνηκε να βοηθάει στην βελτιστοποίηση των επιδόσεων.

Παρακάτω θα δούμε αναλυτικότερα την διαδικασία που ακολουθήθηκε στην προσπάθεια βελτιστοποίησης καθενός από τους ζητούμενους classifiers.

### Dummy classifier

Στον Dummy classifier, στο params_dicts, πέρα από τα υπόλοιπα που αναφέρθηκαν έχουμε βάλει ταυτόχρονα στον ίδιο τον classifier να επιλέγει μεταξύ 4 στρατηγικών, και παραλείψαμε την στρατηγική constant, διότι δεν θέλουμε να προβλέπει μια συγκεκριμένη ετικέτα που θα δώσουμε εμείς.

In [ ]:
params_dicts = {
    'robust_scaler': {
        'with_centering': [True, False],
        'with_scaling': [True, False],
        'unit_variance': [True, False],
    },
    'standard_scaler': {
        'with_mean': [True, False],
        'with_std': [True, False]
    },
    'pca': {
        'n_components': [i for i in range(10, 70, 10)]
    },
    'dummy': {
        'strategy': ['stratified', 'most_frequent', 'prior', 'uniform']
    },
    'select_percentile': {
        'percentile': [i for i in range(10, 110, 10)]
    }
}

#### Accuracy based scoring

Συγκεκριμένα τώρα με μετρική το accuracy, εξετάσαμε μια σειρά από περιπτώσεις και συνδυασμούς μεταξύ των προαναφερθέντων operations, με σκοπό να δούμε ποιος θα οδηγήσει σε μεγαλύτερο score. Αρχικά, δοκιμάσαμε μόνο με τον dummy operation, χωρίς να έχουμε μεγάλο score, όπως και περιμέναμε. Στην συνέχεια, με βάση τον dummy πάντα, όταν προσθέσαμε και scaler, αυξήθηκε και στους δύο scalers, αλλά με τον robust να είχε καλύτερο αποτέλεσμα. Ύστερα, αφαιρέσαμε τους scaler και βάλαμε PCA, αλλά οι επιδόσεις ουσιαστικά μειώθηκαν. Όταν στην συνέχεια βάλαμε και PCA και τους δύο scaler εναλλάξ μεταξύ τους, παρατηρήσαμε πως οι επιδόσεις γενικά αυξήθηκαν, με την μέγιστη από όλες να είναι η Standard Scaler + PCA, την οποία και τελικά χρησιμοποιήσαμε, όπως φαίνεται παρακάτω στα operations. Στο τέλος, βάλαμε και το Feature Selection επιπλεόν σε αυτά, αλλά η επίδοση έμεινε ουσιαστικά η ίδια, με ελάχιστη μείωση. Για αυτόν τον λόγο, λοιπόν, κρατάμε το Standard Scaler + PCA σαν την καλύτερη περίπτωση για τον Dummy με μετρική accuracy.

In [ ]:
operations = [('standard_scaler', StandardScaler()), ('pca', PCA()),
              ('dummy', DummyClassifier())]

param_grid = get_params(params_dicts, operations)

dummy_acc_classifier = use_cv(operations, 'accuracy', param_grid, 'Dummy')

#### F1 based scoring

Την ίδια διαδικασία ακολουθήσαμε και για την μετρική του F1. Αρχικά, δοκιμάσαμε μόνο με τον dummy operation, χωρίς να έχουμε μεγάλο score, όπως και περιμέναμε. Στην συνέχεια, με βάση τον dummy πάντα, όταν προσθέσαμε και scaler, στον Standard μειώθηκε ενώ στον Robust έμεινε περίπου ίδιο. Περίπου ίδιο παρέμεινε και όταν αφαιρέσαμε τους scaler και βάλαμε μόνο το PCA. Στην συνέχεια, όταν βάλαμε και PCA και τους δύο scaler εναλλάξ, παρατήρησαμε ότι τα αποτελέσματα ήταν ξανά περίπου ίδια, με μια μικρή αύξηση. Στο τέλος, δοκιμάζοντας τόσο τους scaler, όσο και το Feature Selection και το PCA μαζί, τότε παρατηρήσαμε ότι σημειώθηκε αύξηση, με την μεγαλύτερη να σημειώνεται στο συνδυασμό Standard Scaler + Feature Selection + PCA, τον οποίο και τελικά χρησιμοποιήσαμε. Εδώ αξίζει να σημειωθεί πως o Dummy Classifier είχε διαφορετικό συνδυασμό για βελτιστοποίηση με μετρική το accuracy, και διαφορετικό με μετρική το F1.

In [ ]:
operations = [('standard_scaler', StandardScaler()), ('select_percentile', SelectPercentile()), ('pca', PCA()),
              ('dummy', DummyClassifier())]

param_grid = get_params(params_dicts, operations)

dummy_f1_classifier = use_cv(operations, 'f1', param_grid, 'Dummy')

### Gaussian Naive Bayes (GNB) classifier

Στον Gaussian Naive Bayes (GNB) classifier, στο params_dicts, πέρα από τα υπόλοιπα που αναφέρθηκαν έχουμε βάλει ταυτόχρονα στον ίδιο τον classifier την παράμετρο var_smoothing.

In [ ]:
params_dicts = {
    'robust_scaler': {
        'with_centering': [True, False],
        'with_scaling': [True, False],
        'unit_variance': [True, False],
    },
    'standard_scaler': {
        'with_mean': [True, False],
        'with_std': [True, False]
    },
    'pca': {
        'n_components': [i for i in range(10, 70, 10)]
    },
    'select_percentile': {
        'percentile': [i for i in range(10, 110, 10)]
    },
    'gnb': {
        'var_smoothing': np.logspace(0, -9, 50)
    }
}

#### Accuracy based scoring

Την ίδια νοοτροπία ακολουθήσαμε και για τον GNB με μετρική accuracy. Αρχικά, δοκιμάσαμε μόνο με το gnb operation, χωρίς να έχουμε ιδιαίτερα μεγάλο score, όπως και περιμέναμε. Στην συνέχεια, με βάση τον gnb πάντα, όταν προσθέσαμε και scaler, δεν παρατηρήσαμε αύξηση σε κανένα από τους 2. Μεγάλη αύξηση παρατηρήσαμε όταν βγάλαμε τους scaler και βάλαμε μόνο το PCA. Στην συνέχεια, όταν μαζί με τον PCA βάλαμε και τους δύο scaler εναλλάξ, τότε παρατηρήσαμε πως υπήρχε μια μικρή βελτίωση αναλογικά με το PCA μόνο του, και για αυτό θεωρήσαμε και τον συνδυασμό Standard Scaler + PCA ως τον καλύτερο, τον οποίο και χρησιμοποιήσαμε κιόλας, όπως φαίνεται παρακάτω. Τέλος, όταν προσθέσαμε και Feature Selection μαζί με τα άλλα δύο που είχαμε πριν παρατηρούμε πως έβγαινε περίπου το ίδιο αποτέλεσμα, χωρίς ιδιαίτερη αλλαγή, οπότε και για αυτό αποφασίσαμε να χρησιμοποιήσουμε τον συνδυασμό Standard Scaler + PCA ως τον βέλτιστο.

In [ ]:
operations = [('standard_scaler', StandardScaler()), ('pca', PCA()), ('gnb', GaussianNB())]

param_grid = get_params(params_dicts, operations)

gnb_acc_classifier = use_cv(operations, 'accuracy', param_grid,
                            'Gaussian Naive Bayes')

#### F1 based scoring

Όπως και πριν, για μετρική F1 τώρα, αρχικά δοκιμάσαμε μόνο με το gnb operation, χωρίς να έχουμε ιδιαίτερα μεγάλο score, όπως και περιμέναμε. Στην συνέχεια, όταν βάλαμε τους δύο scaler εναλλάξ, παρατήρησαμε πως τα αποτελέσματα ήταν ουσιαστικά τα ίδια, με μια μικρή αύξηση. Όταν βγάλαμε τους scaler και βάλαμε το PCA μόνο του, τότε παρατηρήσαμε αύξηση περισσότερη από την αύξηση που είχαν προκαλέσει οι scaler μόνοι τους. Έτσι, όταν στην συνέχεια βάλαμε και το PCA και τους δύο scaler εναλλάξ, πήραμε την μέγιστη βελτίωση, την οποία και τελικά κρατήσαμε, στον συνδυασμό Standard Scaler + PCA. Στο τέλος, όταν βάλαμε επιπλέον και Feature Selection σε αυτά που είχαμε ήδη, τότε παρατηρήσαμε αποτέλεσμα ουσιαστικά ίδιο με το βέλτιστο, με μια μικρή μείωση, οδηγώντας μας έτσι στο συμπέρασμα πως στο GNB για την μετρική F1 ο βέλτιστος συνδυασμός είναι ο Standard Scaler + PCA, όπως ήταν και για την μετρική accuracy αντίστοιχα.

In [ ]:
operations = [('standard_scaler', StandardScaler()), ('pca', PCA()), ('gnb', GaussianNB())]

param_grid = get_params(params_dicts, operations)

gnb_f1_classifier = use_cv(operations, 'f1', param_grid, 'Gaussian Naive Bayes')

### KNeirestNeighbors (kNN) classifier

Για την βελτιστοποίηση του KNeirestNeighbors (kNN) classifier, στο params_dicts, πέρα από αυτά που έχουν αναφερθεί χρησιμοποιήσαμε στον ίδιο τον classifier σαν υπερπαραμέτρους το n_neighbors, που είναι μια λίστα από την οποία επιλέγουμε τον "βέλτιστο" αριθμό από τους κοντινότερους γείτονες, το weights, που είναι η συνάρτηση βάρους που χρησιμοποιείται στην πρόβλεψη, καθώς και το metric, που είναι η μετρική της απόστασης. Επίσης, στον knn τον ίδιο χρησιμοποιήσαμε και την παράμετρο n_jobs ίση με -1, έτσι ώστε να χρησιμοποιηθούν όλοι οι processors στην αναζήτηση των γειτόνων, όπως φαίνεται παρακάτω.

In [ ]:
params_dicts = {
    'robust_scaler': {
        'with_centering': [True, False],
        'with_scaling': [True, False],
        'unit_variance': [True, False],
    },
    'standard_scaler': {
        'with_mean': [True, False],
        'with_std': [True, False]
    },
    'pca': {
        'n_components': [i for i in range(10, 70, 10)]
    },
    'select_percentile': {
        'percentile': [i for i in range(10, 110, 10)]
    },
    'knn': {
        'n_neighbors': list(range(1, 31, 2)),
        'weights': ["uniform", "distance"],
        'metric': ["euclidean", "manhattan", "minkowski"]
    }
}

#### Accuracy based scoring

Με την ίδια λογική με πριν, για την μετρική accuracy του KNN αρχικά δοκιμάσαμε μόνο με το knn operation, στο οποίο παρατηρήσαμε γενικά μεγάλο score. Στην συνέχεια, με βάση πάντα το knn operation, όταν βάλαμε τους δύο scaler εναλλάξ, παρατηρήσαμε πως το αποτέλεσμα ήταν περίπου ίδιο, με μια μικρή αύξηση. Ύστερα, όταν βγάλαμε τους scalers και βάλαμε το PCA μόνο του παρατηρήσαμε μια μικρή μείωση. Ωστόσο, όταν μετά βάλαμε και το PCA αλλά και τους δύο scaler εναλλάξ, παρατηρήσαμε πως είχαμε αύξηση του αποτελέσματος αναλογικά με το αποτέλεσμα που έδινε το PCA μόνο του. Τέλος, όταν στον συνδυασμό αυτό προσθέσαμε και το Feature Selection, τότε παρατηρήσαμε πως η επίδοση ανέβηκε ακόμα περισσότερο από ότι πριν, με αποτέλεσμα να θεωρούμε τον συνδυασμό Standard Scaler + Feature Selection + PCA ως τον βέλτιστο για τον KNN με την μετρική accuracy.

In [ ]:
operations = [('standard_scaler', StandardScaler()),
              ('select_percentile', SelectPercentile()), ('pca', PCA()),
              ('knn', KNeighborsClassifier(n_jobs=-1))]

param_grid = get_params(params_dicts, operations)

knn_acc_classifier = use_cv(operations, 'accuracy', param_grid,
                            'K-nearest neighbors')

#### F1 based scoring

Όμοια με πριν, για την μετρική F1 τώρα, αρχικά δοκιμάσαμε μόνο με το knn operation, στο οποίο παρατηρήσαμε γενικά μεγάλο score. Στην συνέχεια, με βάση πάντα το knn operation, όταν βάλαμε τους δύο scaler εναλλάξ, παρατηρήσαμε πως το αποτέλεσμα ήταν περίπου ίδιο, με μια μικρή αύξηση. Ύστερα, όταν βγάλαμε τους scalers και βάλαμε το PCA μόνο του παρατηρήσαμε μια μικρή μείωση. Ωστόσο, όταν μετά βάλαμε και το PCA αλλά και τους δύο scaler εναλλάξ, παρατηρήσαμε ότι στον Standard είχαμε αποτέλεσμα καλύτερο από το αρχικό score, ενώ στον Robust είχαμε ελαφρώς μικρότερο. Τέλος, όταν στον συνδυασμό αυτό προσθέσαμε και το Feature Selection, τότε παρατηρήσαμε πως η επίδοση ανέβηκε το περισσότερο που είχε ανέβει από όλες τις περιπτώσεις, με αποτέλεσμα να θεωρούμε τον συνδυασμό Standard Scaler + Feature Selection + PCA ως τον βέλτιστο για τον KNN με την μετρική F1, όπως ακριβώς συνέβη και στην μετρική accuracy.

In [ ]:
operations = [('standard_scaler', StandardScaler()),
              ('select_percentile', SelectPercentile()), ('pca', PCA()),
              ('knn', KNeighborsClassifier(n_jobs=-1))]

param_grid = get_params(params_dicts, operations)

knn_f1_classifier = use_cv(operations, 'f1', param_grid, 'K-nearest neighbors')

### Logistic Regression (LR) classifier

Για την βελτιστοποίηση του Logistic Regression (LR) classifier, στο params_dict, πέρα από αυτά που αναφέρθηκαν στην αρχή, χρησιμοποιήσαμε σαν υπερπαραμέτρους του ίδιου του classifier το penalty, το οποίο παίρνει μια από τις παρακάτω 3 τιμές, το fit_intercept, για το αν μια σταθερά πρέπει να προστεθεί στην συνάρτηση απόφασης, καθώς και το C, που παίρνει τιμές από μια λίστα, όπως φαίνεται παρακάτω. Επίσης, στον ίδιο τον classifier χρησιμοποιήσαμε την παράμετρο n_jobs ίση με -1, έτσι ώστε να χρησιμοποιηθούν όλοι οι processors στην διαδικασία, καθώς και τον solver saga, όπως παρουσιάζεται παρακάτω.

In [ ]:
params_dicts = {
    'robust_scaler': {
        'with_centering': [True, False],
        'with_scaling': [True, False],
        'unit_variance': [True, False],
    },
    'standard_scaler': {
        'with_mean': [True, False],
        'with_std': [True, False]
    },
    'pca': {
        'n_components': [i for i in range(10, 70, 10)]
    },
    'select_percentile': {
        'percentile': [i for i in range(10, 110, 10)]
    },
    'lr': {
        'penalty': ["l1", "l2", "elasticnet"],
        'fit_intercept': [True, False],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    }
}

#### Accuracy based scoring

Αντίστοιχα με την διαδικασία που ακολουθήσαμε στις προηγούμενες περιπτώσεις, έτσι και εδώ, αρχικά δοκιμάσαμε μόνο με το lr operation, στο οποίο παρατηρήσαμε γενικά καλό score. Στην συνέχεια, με βάση πάντα το lr operation, όταν βάλαμε τους δύο scaler εναλλάξ, παρατηρήσαμε πως το αποτέλεσμα ήταν περίπου ίδιο, με μια μικρή αύξηση. Όταν στην συνέχεια αφαιρέσαμε τους scaler και βάλαμε μόνο PCA, τότε το αποτέλεσμα αυξήθηκε ακόμα περισσότερο. Ύστερα, όταν μαζί με το PCA βάλαμε εναλλάξ τους scaler, τότε παρατηρήσαμε πως το αποτέλεσμα δεν άλλαξε ιδιαίτερα. Τέλος, με την προσθήκη του Feature Selection στον παραπάνω συνδυασμό, το αποτέλεσμα έμεινε ίδιο με μια αύξηση που ουσιαστικά ήταν και η μέγιστη που παρατηρήσαμε, με αποτέλεσμα να θεωρούμε τον συνδυασμό Standard Scaler + Feature Selection + PCA ως τον βέλτιστο για το Logistic Regression με μετρική accuracy.

In [ ]:
operations = [('standard_scaler', StandardScaler()), ('select_percentile', SelectPercentile()), ('pca', PCA()),
              ('lr', LogisticRegression(n_jobs=-1, solver="saga"))]

param_grid = get_params(params_dicts, operations)

lr_acc_classifier = use_cv(operations, 'accuracy', param_grid,
                           'Logistic Regression')

#### F1 based scoring

Αντίστοιχα με την διαδικασία που ακολουθήσαμε πριν, για το Logistic Regression με μετρική F1 αρχικά δοκιμάσαμε πάλι μόνο με το lr operation, στο οποίο παρατηρήσαμε γενικά καλό score. Στην συνέχεια, με βάση πάντα το lr operation, όταν βάλαμε τους δύο scaler εναλλάξ, παρατηρήσαμε πως το αποτέλεσμα παρουσίασε μια μικρή αύξηση. Όταν στην συνέχεια αφαιρέσαμε τους scaler και βάλαμε μόνο PCA, τότε το αποτέλεσμα αυξήθηκε ακόμα περισσότερο. Ύστερα, όταν μαζί με το PCA βάλαμε εναλλάξ τους scaler, τότε παρατηρήσαμε πως το αποτέλεσμα δεν άλλαξε ιδιαίτερα. Τέλος, με την προσθήκη του Feature Selection στον παραπάνω συνδυασμό, το αποτέλεσμα έμεινε ίδιο με μια αύξηση που ουσιαστικά ήταν και η μέγιστη που παρατηρήσαμε, με αποτέλεσμα να θεωρούμε τον συνδυασμό Standard Scaler + Feature Selection + PCA ως τον βέλτιστο για το Logistic Regression με μετρική F1, όπως ακριβώς συνέβη και στην μετρική accuracy.

In [ ]:
operations = [('standard_scaler', StandardScaler()), ('select_percentile', SelectPercentile()), ('pca', PCA()),
              ('lr', LogisticRegression(n_jobs=-1, solver="saga"))]

param_grid = get_params(params_dicts, operations)

lr_f1_classifier = use_cv(operations, 'f1', param_grid, 'Logistic Regression')

## Εκπαίδευση και test βέλτιστων μοντέλων

In [ ]:
train_best_model(dummy_acc_classifier, 'Dummy', 'accuracy')

In [ ]:
train_best_model(dummy_f1_classifier, 'Dummy', 'f1')

In [ ]:
train_best_model(gnb_acc_classifier, 'Gaussian Naive Bayes', 'accuracy')

In [ ]:
train_best_model(gnb_f1_classifier, 'Gaussian Naive Bayes', 'f1')

In [ ]:
train_best_model(knn_acc_classifier, 'K-nearest neighbors', 'accuracy')

In [ ]:
train_best_model(knn_f1_classifier, 'K-nearest neighbors', 'f1')

In [ ]:
train_best_model(lr_acc_classifier, 'Logistic Regression', 'accuracy')

In [ ]:
train_best_model(lr_f1_classifier, 'Logistic Regression', 'f1')

## Αποτελέσματα και συμπεράσματα

In [ ]:
preprocessing_scores[preprocessing_scores.columns.drop('Preds')].groupby(
    'Classifier').apply(lambda a: a.drop('Classifier', axis=1)[:])

In [ ]:
to_plot_scores = preprocessing_scores[[
    'Classifier', 'Scoring_metric', 'Accuracy', 'F1'
]]

to_plot_differences = preprocessing_scores[[
    'Classifier', 'Scoring_metric', 'ΔAccuracy', 'ΔF1'
]]

In [ ]:
migration = lambda x: {
    'Classifier':
        x,
    'Scoring_metric':
        'Out of the box',
    'Accuracy':
        out_of_the_box_scores.loc[
            (out_of_the_box_scores['Classifier'] == x) &
            (out_of_the_box_scores['Metric'] == 'Accuracy')]['Score'].values[0],
    'F1':
        out_of_the_box_scores.loc[
            (out_of_the_box_scores['Classifier'] == x) &
            (out_of_the_box_scores['Metric'] == 'F1')]['Score'].values[0],
}

old_scores = pd.DataFrame()
for clf in np.unique(out_of_the_box_scores['Classifier']):
    old_scores = old_scores.append(migration(clf), ignore_index=True)
to_plot_scores = old_scores.append(to_plot_scores, ignore_index=True)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20, 20))
axs[0][0].get_shared_y_axes().join(axs[0][0], axs[0][1])
axs[1][0].get_shared_y_axes().join(axs[1][0], axs[1][1])

sns.barplot(
    data=to_plot_scores,
    y='Accuracy',
    x='Classifier',
    hue="Scoring_metric",
    ax=axs[0][0]).set_title("Accuracy of optimized and out of the box models")

sns.barplot(data=to_plot_scores,
            y='F1',
            x='Classifier',
            hue="Scoring_metric",
            ax=axs[0][1]).set_title("F1 of optimized and out of the box models")

sns.barplot(data=to_plot_differences,
            y='ΔAccuracy',
            x='Classifier',
            hue="Scoring_metric",
            palette=["C1", "C2"],
            ax=axs[1][0]).set_title(
                "Accuracy difference of optimized and out of the box models")
sns.barplot(data=to_plot_differences,
            y='ΔF1',
            x='Classifier',
            hue="Scoring_metric",
            palette=["C1", "C2"],
            ax=axs[1][1]).set_title(
                "F1 difference of optimized and out of the box models")

plt.plot()

In [ ]:
without_dummy = preprocessing_scores.loc[
    preprocessing_scores['Classifier'] != 'Dummy']
worst_model = without_dummy[without_dummy['Accuracy'] ==
                            without_dummy['Accuracy'].min()].iloc[0]
best_model = without_dummy[without_dummy['Accuracy'] ==
                           without_dummy['Accuracy'].max()].iloc[0]

print('''Worst model is {} with scoring metric {} and accuracy {:.3f}
Best model is {} with scoring metric {} and accuracy {:.3f}'''.format(
    worst_model['Classifier'], worst_model['Scoring_metric'],
    worst_model['Accuracy'], best_model['Classifier'],
    best_model['Scoring_metric'], best_model['Accuracy']))

In [ ]:
worst_cf_matrix = confusion_matrix(worst_model['Preds'], test_labels)
best_cf_matrix = confusion_matrix(best_model['Preds'], test_labels)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(20, 8))

group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in worst_cf_matrix.flatten()]
group_percentages = [
    '{0:.2%}'.format(value)
    for value in worst_cf_matrix.flatten() / np.sum(worst_cf_matrix)
]
labels = [
    f'{v1}\n{v2}\n{v3}'
    for v1, v2, v3 in zip(group_names, group_counts, group_percentages)
]
labels = np.asarray(labels).reshape(2, 2)
sns.heatmap(worst_cf_matrix, annot=labels, fmt='', cmap='Blues',
            ax=axs[0]).set_title('Worst model confussion matrix')

group_counts = ['{0:0.0f}'.format(value) for value in best_cf_matrix.flatten()]
group_percentages = [
    '{0:.2%}'.format(value)
    for value in best_cf_matrix.flatten() / np.sum(best_cf_matrix)
]
labels = [
    f'{v1}\n{v2}\n{v3}'
    for v1, v2, v3 in zip(group_names, group_counts, group_percentages)
]
labels = np.asarray(labels).reshape(2, 2)
sns.heatmap(best_cf_matrix, annot=labels, fmt='', cmap='Blues',
            ax=axs[1]).set_title('Best model confussion matrix')

plt.show()